In [1]:
#Optimizacion Bayesiana de hiperparametros de  rpart

#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

library("data.table")
library("rlist")

library("rpart")
library("parallel")

#paquetes necesarios para la Bayesian Optimization
library("DiceKriging")
library("mlrMBO")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,614757,32.9,1321401,70.6,1321401,70.6
Vcells,1093689,8.4,8388608,64.0,1632451,12.5


Warning message:
"package 'data.table' was built under R version 4.2.3"
Warning message:
"package 'rlist' was built under R version 4.2.3"
Warning message:
"package 'rpart' was built under R version 4.2.3"
Warning message:
"package 'DiceKriging' was built under R version 4.2.3"
Warning message:
"package 'mlrMBO' was built under R version 4.2.3"
Loading required package: mlr

Warning message:
"package 'mlr' was built under R version 4.2.3"
Loading required package: ParamHelpers

Warning message:
"package 'ParamHelpers' was built under R version 4.2.3"
Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: smoof

Warning message:
"package 'smoof' was built under R version 4.2.3"
Loading required package: checkmate

Warning message:
"package 'checkmate' was built under R

In [2]:
#Defino la  Optimizacion Bayesiana

kBO_iter  <- 100   #cantidad de iteraciones de la Optimizacion Bayesiana

hs  <- makeParamSet(
          makeNumericParam("cp"       , lower= -1   , upper=    0.1),
          makeIntegerParam("minsplit" , lower=  1L  , upper= 8000L),  #la letra L al final significa ENTERO
          makeIntegerParam("minbucket", lower=  1L  , upper= 4000L),
          makeIntegerParam("maxdepth" , lower=  3L  , upper=   20L),
          forbidden = quote( minbucket > 0.5*minsplit ) )             # minbuket NO PUEDE ser mayor que la mitad de minsplit

ksemilla_azar  <- 262139   #cambiar por la primer semilla

In [3]:
#graba a un archivo los componentes de lista
#para el primer registro, escribe antes los titulos

loguear  <- function( reg, arch=NA, folder="./work/", ext=".txt", verbose=TRUE )
{
  archivo  <- arch
  if( is.na(arch) )  archivo  <- paste0( folder, substitute( reg), ext )

  if( !file.exists( archivo ) )  #Escribo los titulos
  {
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  linea  <- paste0( format(Sys.time(), "%Y%m%d %H%M%S"),  "\t",     #la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  #grabo al archivo

  if( verbose )  cat( linea )   #imprimo por pantalla
}

In [4]:
#particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 
# particionar( data=dataset, division=c(1,1,1,1,1), agrupa=clase_ternaria, seed=semilla)   divide el dataset en 5 particiones

particionar  <- function( data, division, agrupa="", campo="fold", start=1, seed=NA )
{
  if( !is.na( seed)  )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x ) }, division, seq( from=start, length.out=length(division) )  ) )

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
           by= agrupa ]
}

In [5]:
#fold_test  tiene el numero de fold que voy a usar para testear, entreno en el resto de los folds
#param tiene los hiperparametros del arbol

ArbolSimple  <- function( fold_test, data, param )
{
  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .", 
                   data= data[ fold != fold_test, ],  #entreno en todo MENOS el fold_test que uso para testing
                   xval= 0,
                   control= param )

  #aplico el modelo a los datos de testing
  prediccion  <- predict( modelo, 
                          data[ fold==fold_test, ],  #aplico el modelo sobre los datos de testing
                          type= "prob")   #quiero que me devuelva probabilidades

  prob_baja2  <- prediccion[, "BAJA+2"]  #esta es la probabilidad de baja

  #calculo la ganancia
  ganancia_testing  <- data[ fold==fold_test ][ prob_baja2 > 1/40,  
                                                sum( ifelse( clase_ternaria=="BAJA+2", 117000, -3000 ) )] 

  return( ganancia_testing )  #esta es la ganancia sobre el fold de testing, NO esta normalizada
}

In [6]:
ArbolesCrossValidation  <- function( data, param, qfolds, pagrupa, semilla )
{
  divi  <- rep( 1, qfolds )  # generalmente  c(1, 1, 1, 1, 1 )  cinco unos

  particionar( data, divi, seed=semilla, agrupa=pagrupa )  #particiono en dataset en folds

  ganancias  <- mcmapply( ArbolSimple, 
                          seq(qfolds), # 1 2 3 4 5
                          MoreArgs= list( data, param), 
                          SIMPLIFY= FALSE,
                          mc.cores= 1 )   #se puede subir a qfolds si posee Linux o Mac OS

  data[ , fold := NULL ]

  #devuelvo la primer ganancia y el promedio
  ganancia_promedio  <- mean( unlist( ganancias ) )   #promedio las ganancias
  ganancia_promedio_normalizada  <- ganancia_promedio * qfolds  #aqui normalizo la ganancia
  
  return( ganancia_promedio_normalizada )
}

In [7]:
#esta funcion solo puede recibir los parametros que se estan optimizando
#el resto de los parametros, lamentablemente se pasan como variables globales

EstimarGanancia  <- function( x )
{
   GLOBAL_iteracion  <<-  GLOBAL_iteracion + 1

   xval_folds  <- 5
   ganancia  <- ArbolesCrossValidation( dataset,
                                        param= x, #los hiperparametros del arbol
                                        qfolds= xval_folds,  #la cantidad de folds
                                        pagrupa= "clase_ternaria",
                                        semilla= ksemilla_azar )

   #logueo 
   xx  <- x
   xx$xval_folds  <-  xval_folds
   xx$ganancia  <- ganancia
   xx$iteracion <- GLOBAL_iteracion
   loguear( xx,  arch= archivo_log )

   return( ganancia )
}

In [8]:
#Aqui empieza el programa

setwd(".\\")   #Establezco el Working Directory

#cargo los datos
dataset  <- fread("./datasets/dataset_pequeno.csv")
#trabajo solo con los datos con clase, es decir 202107
dataset  <- dataset[ clase_ternaria!= "" ]

In [9]:
#creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( "./exp/HT3210/", showWarnings = FALSE )
setwd("./exp/HT3210/")   #Establezco el Working Directory DEL EXPERIMENTO

archivo_log  <- "HT321.txt"
archivo_BO   <- "HT321.RDATA"

In [10]:
#leo si ya existe el log, para retomar en caso que se se corte el programa
GLOBAL_iteracion  <- 0

if( file.exists(archivo_log) )
{
 tabla_log  <- fread( archivo_log )
 GLOBAL_iteracion  <- nrow( tabla_log )
}

In [11]:
#Aqui comienza la configuracion de la Bayesian Optimization

funcion_optimizar  <- EstimarGanancia

configureMlr( show.learner.output= FALSE)



In [12]:
#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar,
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,
              has.simple.signature = FALSE
             )

ctrl  <- makeMBOControl( save.on.disk.at.time= 600,  save.file.path= archivo_BO)
ctrl  <- setMBOControlTermination(ctrl, iters= kBO_iter )
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km  <- makeLearner("regr.km", predict.type= "se", covtype= "matern3_2", control= list(trace= TRUE))



In [13]:
#inicio la optimizacion bayesiana
if( !file.exists( archivo_BO ) ) {

  run  <- mbo( fun=     obj.fun, 
               learner= surr.km,
               control= ctrl)

} else  run  <- mboContinue( archivo_BO )   #retomo en caso que ya exista



Computing y column(s) for design. Not provided.



20230413 032846	-0.769594018842326	4039	889	14	5	52770000	1
20230413 032919	-0.110603534347319	3053	65	8	5	51924000	2
20230413 032950	-0.0282520969645701	7282	3244	11	5	49659000	3
20230413 033012	-0.447911744035082	7872	3575	5	5	49452000	4
20230413 033050	-0.219489627087023	4808	2119	20	5	53094000	5
20230413 033110	-0.855873545749637	3613	1101	3	5	50340000	6
20230413 033153	-0.908433662202151	5959	733	17	5	50640000	7
20230413 033239	-0.584481969926856	1559	331	13	5	50466000	8
20230413 033302	-0.062879240750044	6968	1377	5	5	50328000	9
20230413 033356	-0.39795836668262	814	72	15	5	50505000	10
20230413 033435	-0.973762301259142	3801	573	12	5	52137000	11
20230413 033451	-0.645764183163244	7560	1149	3	5	50340000	12
20230413 033509	0.0816108253119248	5992	2606	8	5	0	13
20230413 033544	-0.952528139863474	5585	2598	11	5	51165000	14
20230413 033608	-0.0411513211904093	2054	10	5	5	53163000	15
20230413 033653	-0.625215984112583	6462	341	18	5	49995000	16


[mbo] 0: cp=-0.77; minsplit=4039; minbucket=889; maxdepth=14 : y = 5.28e+07 : 42.2 secs : initdesign

[mbo] 0: cp=-0.111; minsplit=3053; minbucket=65; maxdepth=8 : y = 5.19e+07 : 32.6 secs : initdesign

[mbo] 0: cp=-0.0283; minsplit=7282; minbucket=3244; maxdepth=11 : y = 4.97e+07 : 31.1 secs : initdesign

[mbo] 0: cp=-0.448; minsplit=7872; minbucket=3575; maxdepth=5 : y = 4.95e+07 : 22.4 secs : initdesign

[mbo] 0: cp=-0.219; minsplit=4808; minbucket=2119; maxdepth=20 : y = 5.31e+07 : 37.9 secs : initdesign

[mbo] 0: cp=-0.856; minsplit=3613; minbucket=1101; maxdepth=3 : y = 5.03e+07 : 20.0 secs : initdesign

[mbo] 0: cp=-0.908; minsplit=5959; minbucket=733; maxdepth=17 : y = 5.06e+07 : 42.8 secs : initdesign

[mbo] 0: cp=-0.584; minsplit=1559; minbucket=331; maxdepth=13 : y = 5.05e+07 : 45.9 secs : initdesign

[mbo] 0: cp=-0.0629; minsplit=6968; minbucket=1377; maxdepth=5 : y = 5.03e+07 : 23.0 secs : initdesign

[mbo] 0: cp=-0.398; minsplit=814; minbucket=72; maxdepth=15 : y = 5.05e+

20230413 033729	-0.751181858466254	4472	2069	20	5	52833000	17


[mbo] 1: cp=-0.751; minsplit=4472; minbucket=2069; maxdepth=20 : y = 5.28e+07 : 34.8 secs : infill_ei



...

[mbo] 98: cp=-0.341; minsplit=1571; minbucket=2; maxdepth=14 : y = 5.47e+07 : 42.3 secs : infill_ei



20230413 042716	-0.459614912912468	56	17	16	5	38715000	115


[mbo] 99: cp=-0.46; minsplit=56; minbucket=17; maxdepth=16 : y = 3.87e+07 : 49.8 secs : infill_ei



20230413 042743	-0.436932407765606	3043	1518	8	5	52938000	116


[mbo] 100: cp=-0.437; minsplit=3043; minbucket=1518; maxdepth=8 : y = 5.29e+07 : 25.3 secs : infill_ei

Saved the final state in the file HT321.RDATA

